# Ejercicios Unidad 1

5. (Entrega obligatoria individual en repo) Genere un video en un patio o en un hall de edificio donde en un principio se vea vacío y luego aparezca una persona. Mediante los métodos de motion detection (sin usar deep learning) logre una detección de la persona cuando entra al cuadro suponiendo la utilidad para una cámara de seguridad.
Luego sobre el mismo video aplique los algoritmos de flujo denso y disperso que se mostraron en clase.
Escriba una reflexión sobre los resultados en el formato md dentro del Jupyter Notebook.
6. (Entrega obligatoria individual en repo) Explique cuál es diferencia entre localización de objetos y clasificación de imágenes. Muestre ejemplos de ello.


## Instalar e importar librerias

In [ ]:
!pip install mediapy ultralytics ffmpeg

In [4]:
# Importar librerias para procesar videos e imagenes
import cv2

# Importar librerias para graficar
import matplotlib.pyplot as plt

# Importar librerias para manipulacion de imagenes
import numpy as np

# Importar librerias para visualizar y modificar videos
import mediapy as media
import ffmpeg

# Importar librerias para utilizar modelo de deteccion de objetos
from ultralytics import YOLO


## Definir funciones de procesamientos de video

### Funcion para leer el video y aplicar procesamiento

In [8]:
# Función para procesar un video:
def video_processor(filename_in, filename_out, process_func, max_time=10, **kwargs):
    # Abrir el video de entrada para lectura
    with media.VideoReader(filename_in) as r:
        # Crear un archivo de video de salida
        with media.VideoWriter(filename_out, shape=r.shape, fps=r.fps, bps=r.bps) as w:
            count = 0  # Inicializar contador de fotogramas
            prev_image = None  # Inicializar la imagen previa

            # Iterar sobre cada imagen (fotograma) del video
            for image in r:
                new_image = media.to_uint8(image)  # Convertir la imagen a formato flotante

                # Comprobar si es la primera imagen
                if prev_image is None:
                    prev_image = new_image.copy()

                # Procesar la imagen utilizando la función dada
                processed_image = process_func(new_image, prev_image, **kwargs)

                # Añadir la imagen procesada al video de salida
                w.add_image(processed_image)

                # Actualizar la imagen previa
                prev_image = new_image.copy()

                # Incrementar el contador de fotogramas
                count += 1

                # Detener el proceso si se alcanza el tiempo máximo
                if count >= max_time * r.fps:
                    break

### Funcion para dibujar los contornos en el video

In [ ]:
def draw_contours(frame, contours, color=(0, 255, 0), thickness=2):
    # Comprobar si la imagen es en escala de grises (1 canal)
    if len(frame.shape) == 2 or frame.shape[2] == 1:
        # Convertir la imagen de escala de grises a color (3 canales)
        result_image = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    else:
        # Si ya es una imagen de color, simplemente hacer una copia
        result_image = frame.copy()

    # Dibujar cada contorno en la imagen
    for contour in contours:
        # Obtener el rectángulo delimitador para cada contorno
        x, y, w, h = cv2.boundingRect(contour)
        # Dibujar el rectángulo
        cv2.rectangle(result_image, (x, y), (x + w, y + h), color, thickness)

    return result_image

### Funcion para la deteccion de objetos

In [9]:
# Función actualizada para detectar movimientos y dibujar cuadros delimitadores:
def process_frame_difference_full(new_image, prev_image, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 0, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 30, 255, cv2.THRESH_BINARY)

    # Dilatar la imagen umbralizada para mejorar la detección de contornos
    kernel = np.ones((5,5),np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations = 1)

    # Convertir la imagen dilatada a formato adecuado para findContours
    dilated = dilated.astype(np.uint8)

    # Encontrar contornos en la imagen dilatada
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Dibujar cuadros delimitadores alrededor de los contornos
    if kwargs.get('draw_mode', 0) == 0:
      result_image = draw_contours(new_image, contours)
    elif kwargs.get('draw_mode', 0) == 1:
      result_image = draw_contours(thresh, contours)

    return result_image

### Funcion para la deteccion de objetos con filtro de contornos

In [28]:
# Función actualizada para detectar movimientos y dibujar cuadros delimitadores
def process_frame_difference_filtro(new_image, prev_image, min_area=3000, **kwargs):
    # Convertir las imágenes a escala de grises
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_RGB2GRAY)
    prev_gray = cv2.cvtColor(prev_image, cv2.COLOR_RGB2GRAY)

    # Calcular la diferencia absoluta entre los fotogramas actual y anterior
    frame_diff = cv2.absdiff(new_gray, prev_gray)

    # Normalizar la imagen de diferencia
    norm_diff = cv2.normalize(frame_diff, None, 50, 255, cv2.NORM_MINMAX)

    # Umbralizar la imagen para resaltar las diferencias
    _, thresh = cv2.threshold(norm_diff, 80, 255, cv2.THRESH_BINARY)

    # Dilatar la imagen umbralizada para mejorar la detección de contornos
    kernel = np.ones((5,5),np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)

    # Convertir la imagen dilatada a formato adecuado para findContours
    dilated = dilated.astype(np.uint8)

    # Encontrar contornos en la imagen dilatada
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar los contornos por área
    filtered_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]


    # Dibujar cuadros delimitadores alrededor de los contornos filtrados
    if kwargs.get('draw_mode', 0) == 0:
        result_image = draw_contours(new_image, filtered_contours)
    elif kwargs.get('draw_mode', 0) == 1:
        result_image = draw_contours(thresh, filtered_contours)

    return result_image

### Funcion para la deteccion de objetos con modelo deep learning

In [14]:
# Load the pre-trained YOLOv8 model
model = YOLO('yolov8n')  # This will automatically download the model weights

# Función para detectar objetos en una imagen:
def detect_objects(new_image, prev_image, **kwargs):
    # Convertir la imagen a float32
    results = model(new_image)

    # Clases de interés para graficar
    classes = kwargs.get('classes', ['person'])

    # Iteramos sobre los boung boxes obtenidos
    for box in results[0].boxes:
        # Extrayendo los datos del tensor
        x1, y1, x2, y2, confidence, cls = box.data[0]

        # Obteniendo el nombre de la clase
        class_name = model.names[int(cls)]

        # Parámetros opcionales del bounding box
        color = kwargs.get('color', (0, 255, 0))
        thickness = kwargs.get('thickness', 2)

        if class_name in classes:
            # Dibujar el rectángulo
            cv2.rectangle(new_image, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)

            # Agregar el texto de la confianza
            confidence_text = f"{class_name}: {confidence:.2f}"
            cv2.putText(new_image, confidence_text, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return new_image


## Usar video sin posicion fija

In [6]:
video_base = '/content/20240408_104646.mp4'

### Deteccion de objeto simple

#### Solo contornos blanco y negro

In [ ]:
video_salida = 'personFrame1.mp4'


# Llamar a la función para procesar el video
video_processor(video_base, video_salida, process_frame_difference_full,
                max_time=10, draw_mode=1)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

#### Aplicado a la imagen a color

In [ ]:
# Llamar a la función para procesar el video
video_salida = 'personFrame0.mp4'

video_processor(video_base, video_salida, process_frame_difference_full,
                max_time=10, draw_mode=0)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

#### Deteccion de objetos con filtro de contornos

In [ ]:
# Llamar a la función para procesar el video
video_salida = 'fullPersonFiltro.mp4'

video_processor(video_base, video_salida, process_frame_difference_filtro,
                max_time=10, draw_mode=0)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

In [ ]:

YOLO_person = 'person_YOLO.mp4'

parameters = dict(classes=['person'])    # ['bus', 'car', 'truck']

# Llamar a la función para procesar el video
video_processor(video_base, YOLO_person, detect_objects, **parameters)

# Mostrar el video resultante
media.show_video(media.read_video(YOLO_person), fps=30)

# Conclusiones de la deteccion de objetos con video a mano
## Deteccion de objetos simple
Al aplicar las funciones de detección de objetos en un video sin una posición fija (cámara en mano), se detectan muchos contornos innecesarios que únicamente "ensucian la detección de los objetos importantes". Esto se puede corregir aplicando un filtro exhaustivo de contornos que estén dentro de un rango de valores específico y ajustando el umbralado, además de emplear otras técnicas de filtrado más avanzadas. Aun así, el algoritmo sigue detectando contornos que no corresponden a la persona, lo que lo hace ineficiente en este tipo de video.

## Deteccion de objetos con modelo de deep learning (extra)
En cuanto al modelo YOLO, al aplicarlo en el video se obtienen resultados excelentes, a pesar de que el video no esté grabado desde una posición fija.

## Usar video con posicion fija

In [ ]:
video_base = '/content/video_fijo.mp4'

#### Solo contornos blanco y negro

In [ ]:
video_salida = 'PRUEBAFrame1.mp4'


# Llamar a la función para procesar el video
video_processor(video_base, video_salida, process_frame_difference_full,
                max_time=10, draw_mode=1)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

#### Deteccion de objetos con filtro de contornos en blanco y negro

In [ ]:
# Llamar a la función para procesar el video
video_salida = 'PRUEBAFrame2.mp4'

video_processor(video_base, video_salida, process_frame_difference_filtro,
                max_time=10, draw_mode=1)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

#### Deteccion de objetos con filtro de contornos en imagen real

In [ ]:
# Llamar a la función para procesar el video
video_salida = 'PRUEBAFrame3.mp4'

video_processor(video_base, video_salida, process_frame_difference_filtro,
                max_time=10, draw_mode=0)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

# Conclusiones de la deteccion de objetos con video fijo
## Deteccion de objetos simple
En este otro caso, se aplicaron las mismas funciones de procesamiento de video para la detección de objetos y el rendimiento fue mucho más óptimo sin aplicar filtros de contornos. Al aplicar los filtros, la detección de objetos es mucho más limpia y casi tan óptima como el modelo YOLO aplicado anteriormente.

# Funciones de flujo denso y disperso

### Funcion de flujo disperso

In [34]:
def process_sparse_optical_flow(new_image, prev_image):
    # Preparamos las imagenes de trabajo
    new_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    prev_gray_image = cv2.cvtColor(prev_image, cv2.COLOR_BGR2GRAY)

    # Verificar si ya se han detectado las características de Shi-Tomasi
    if not hasattr(process_sparse_optical_flow, "shi_tomasi_done"):
        # Definir parámetros para la detección de esquinas de Shi-Tomasi
        feature_params = dict(maxCorners=300, qualityLevel=0.2, minDistance=2, blockSize=7)
        # Detectar puntos característicos en la imagen
        process_sparse_optical_flow.prev_points = cv2.goodFeaturesToTrack(new_gray, mask=None, **feature_params)
        # Crear una máscara para dibujar el flujo óptico
        process_sparse_optical_flow.mask = np.zeros_like(new_image)
        # Marcar que se ha completado la detección de Shi-Tomasi
        process_sparse_optical_flow.shi_tomasi_done = True

    # Continuar si se ha completado la detección de Shi-Tomasi
    if process_sparse_optical_flow.shi_tomasi_done:
        prev_points = process_sparse_optical_flow.prev_points
        mask = process_sparse_optical_flow.mask

    # Parámetros para el flujo óptico de Lucas-Kanade
    lk_params = dict(winSize=(15, 15), maxLevel=2,
                     criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

    # Calcular el flujo óptico de Lucas-Kanade
    new_points, status, error = cv2.calcOpticalFlowPyrLK(prev_gray_image, new_gray, prev_points, None, **lk_params)
    # Filtrar puntos buenos
    good_old = prev_points[status == 1]
    good_new = new_points[status == 1]
    color = (0, 255, 0)  # Color para el dibujo
    # Dibujar el movimiento (flujo óptico)
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.astype(int).ravel()
        c, d = old.astype(int).ravel()
        mask = cv2.line(mask, (a, b), (c, d), color, 2)
        new_image = cv2.circle(new_image, (a, b), 3, color, -1)

    # Combinar la imagen actual con las líneas de flujo óptico dibujadas
    output = cv2.add(new_image, mask)
    # Actualizar puntos para el siguiente cuadro
    process_sparse_optical_flow.prev_points = good_new.reshape(-1, 1, 2)
    return output

### Funcion de flujo denso

In [33]:
# Función para procesar el flujo óptico denso
def process_dense_optical_flow(new_image, prev_image):
    # Convierte la nueva imagen a escala de grises
    gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    if not hasattr(process_dense_optical_flow, "init_done"):
        process_dense_optical_flow.prev_gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
        process_dense_optical_flow.mask = np.zeros_like(new_image)
        process_dense_optical_flow.mask[..., 1] = 255
        process_dense_optical_flow.init_done = True

    if process_dense_optical_flow.init_done:
        prev_gray = process_dense_optical_flow.prev_gray
        mask = process_dense_optical_flow.mask

    # Calcula el flujo óptico
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Computa magnitud y ángulo de los vectores 2D
    magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    # Establece el tono de la imagen según la dirección del flujo óptico
    mask[..., 0] = angle * 180 / np.pi / 2
    # Establece el valor de la imagen según la magnitud del flujo óptico
    mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
    # Convierte de HSV a RGB
    rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
    # Actualiza la imagen previa a gris
    process_dense_optical_flow.prev_grayprev_gray = gray.copy()
    return rgb

## Definir videos base

In [31]:
video_base_MANO = '/content/20240408_104646.mp4'

video_base_FIJO = '/content/video_fijo.mp4'

## Aplicacion de flujo disperso

### Video en mano

In [ ]:
# Nombres de los archivos de video de entrada y salida
video_salida = 'MANO_sparse_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(video_base_MANO, video_salida, process_sparse_optical_flow,
                max_time=10)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

### Video fijo

In [ ]:
# Nombres de los archivos de video de entrada y salida
video_salida = 'FIJO_sparse_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(video_base_FIJO, video_salida, process_sparse_optical_flow,
                max_time=10)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

## Aplicacion de flujo denso

### Video en mano

In [ ]:
# Nombres de los archivos de video de entrada y salida
video_salida = 'MANO_dense_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(video_base_MANO, video_salida, process_dense_optical_flow,
                max_time=20)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

### Video fijo

In [ ]:
# Nombres de los archivos de video de entrada y salida
video_salida = 'FIJO_dense_optical_flow.mp4'

# Llamar a la función para procesar el video
video_processor(video_base_FIJO, video_salida, process_dense_optical_flow,
                max_time=20)

# Mostrar el video resultante
media.show_video(media.read_video(video_salida), fps=30)

# Conclusiones de la aplicacion de funciones de flujo denso y disperso

## Flujo disperso
Al aplicar la función de flujo disperso en ambos videos, funciona de manera clara y se logra ver su aplicación. En ambos videos, hay detecciones de movimiento falsas, pero cuando aparece la persona, se puede visualizar cómo los keypoints "acompañan" la figura de la persona.

## Flujo denso
En cuanto a la aplicación de la función de flujo denso, en ambos videos funciona mal, no siendo clara la visualización del movimiento y generando confusión cuando la persona entra en escena.

El flujo disperso en este caso es el que mejor funciona (entre flujo denso y disperso), siendo clara su aplicación y observando una diferencia de movimiento cuando la persona entra en escena.